## 코드 개요

#### 논문: https://arxiv.org/pdf/2407.20775
#### 깃허브: https://github.com/harryjdavies/HeartGPT

이 코드의 목적은 ECG(심전도) 데이터를 기반으로 트랜스포머 모델을 사전 학습(Pretraining)하는 것이다.
특히 GPT 기반의 트랜스포머를 사용하여 다음 시점(time step)의 신호 값을 예측하는 방식으로 학습이 진행된다.

**모델 구조**
- GPT(Decoder-Only Transformer) 기반
- 8개 어텐션 헤드 (n_head = 8)
- 8개 트랜스포머 블록 (n_layer = 8)
- 맥락 길이(context length) 500
- 임베딩 차원 64 (n_embd = 64)
- 드롭아웃(dropout) 0.2 적용

**학습 데이터**
- .mat 파일(ECG 신호 데이터)에서 불러옴
- 모든 데이터는 0~100 범위로 정규화한 후 101개의 토큰으로 변환
- 90%는 학습(train), 10%는 테스트(test) 데이터로 분할

**손실 함수**

- Cross Entropy Loss: 다음 시점의 신호 값을 예측하는 문제를 다중 분류 문제로 모델링

## 필수 라이브러리

In [ ]:
import torch
import torch.nn as nn  # 신경망 모델을 구성하는 모듈 (nn.Linear, nn.LayerNorm, nn.Dropout 등)
from torch.nn import functional as F  # 활성화 함수(F.softmax), 손실 함수(F.cross_entropy) 등을 포함
import scipy.io  # .mat 파일에서 데이터를 로드하기 위한 라이브러리
import numpy as np
import time  # 성능 측정을 위한 라이브러리

## 하이퍼파라미터 설정

In [ ]:
# Harry Davies 19_09_2024

# The following code is adapted from a tutorial by Andrej Kapathy, available at https://github.com/karpathy/ng-video-lecture
# The explaination behind this code and the model files can be found in the paper "Interpretable Pre-Trained Transformers for Heart Time-Series Data"
# available at https://arxiv.org/abs/2407.20775

eval_interval = 2000
save_interval = 20000 #how often the model is checkpointed
eval_iters = 200
batch_size = 64 # sequences we process in parellel
max_iters = 1000000
block_size = 500 # this is context length
learning_rate = 3e-04
n_embd = 64 # 384 / 6 means every head is 64 dimensional
n_head = 8
n_layer = 8
dropout = 0.2


# GPU is necessary. Training of 8 head, 8 layer model and 500 context length was possible with 12GB VRAM
device = 'cuda' if torch.cuda.is_available() else 'cpu'

- 컨텍스트 길이(block_size = 500)
→ 트랜스포머 모델이 500개 토큰의 신호를 보고 다음 값을 예측
- 어텐션 헤드(n_head = 8) → 8개의 병렬 어텐션을 수행하여 특징을 학습
- 드롭아웃(dropout = 0.2) → 과적합 방지를 위해 20%의 뉴런을 랜덤하게 비활성화
- 최대 학습 횟수(max_iters = 1000000) → 100만 번의 업데이트 수행

## 데이터 로드 및 전처리

In [ ]:
data_load = scipy.io.loadmat('D:/ecg_store_gpt_training') # .mat 파일에서 데이터 불러오기
data_ecg = data_load['ecg_store']

vocab_size = 101  # 신호 값을 0~100 범위의 정수형으로 변환 (101개 토큰)

perm = np.random.permutation(data_ecg.shape[0])  # 데이터 랜덤으로 섞기
data_ecg_rand = data_ecg[perm, :]

data = torch.tensor(data_ecg_rand, dtype=torch.long)

# 90% 학습 데이터, 10% 테스트 데이터로 분할
x_thresh = int(0.9 * data_ecg.shape[0])
train_data = data[:x_thresh, :]
test_data = data[x_thresh:, :]

- ECG 데이터를 정수형 토큰으로 변환하여 트랜스포머에서 학습할 수 있도록 함 
- 90%를 학습 데이터, 10%를 검증 데이터로 사용

## 배치(batch) 데이터 생성

In [ ]:
def get_batch(split):
    data = train_data  if split == 'train' else test_data
    # creates two random indices. One to pick the subject, and one to pick the position in the trace.
    # traces for each subject were never less than 1000 samples. blocksize+ix2 can never be longer than the trace.
    ix = torch.randint(data.shape[0], (batch_size,))
    ix2 = torch.randint(500, (1,))
    x = torch.stack([data[i,ix2:ix2+block_size] for i in ix])
    y = torch.stack([data[i,ix2+1:ix2+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

- 배치 데이터를 무작위로 선택하여 모델에 공급
- x: 현재 시점의 데이터 (입력)
- y: 다음 시점의 데이터 (타겟)
- 모델이 현재 데이터를 보고 다음 시점 데이터를 예측하도록 학습

## 모델 손실 평가 함수

In [ ]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

- @torch.no_grad(): 그래디언트 계산을 비활성화하여 성능 최적화
- 모델을 평가 모드(model.eval())로 설정하여 Dropout 등을 비활성화

## 어텐션(Attention) 구현

In [ ]:
class Head(nn.Module):

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones((block_size,block_size)))) #buffer means not updated by optimiser
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        #start = time.time()
        B, T, C = x.shape
        k = self.key(x)
        q = self.query(x)
        # compute attention scores (affinities)
        wei = q @ k.transpose(-2, -1) * C**-0.5 # square root headsize # (B, T, C) @ (B, C, T) = B, T, T
        # for every batch, we will now have a T by T matrix giving us the affinities of each token
        wei = wei.masked_fill(self.tril[:T,:T] == 0, float('-inf'))
        # the tril signifies a decoder block, future tokens cannot communicate with the past
        wei = F.softmax(wei, dim=-1)# all attention weights sum to 1 for updating a single token
        wei = self.dropout(wei)
        v = self.value(x)
        out = wei @ v
        #end = time.time()
        #print(start-end)
        return out

어텐션 메커니즘 (Q, K, V 행렬 계산) , 어텐션 가중치 계산 및 적용

## 다중 헤드 어텐션 (MultiHeadAttention)

In [ ]:
class MultiHeadAttention(nn.Module):

    def __init__(self, num_heads, head_size):
        super().__init__()
        # creating a list of head objects (turned into modules) resulting in a number of head modules
        # then assigns the list of modules to self.heads - these run in parellel
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd) #projection generally matches sizes for adding in residual connection
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        #concatenate the output of the different attention heads
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

## 피드포워드 신경망 (FeedForward)

In [ ]:
class FeedForward(nn.Module):

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),  # 차원을 4배 증가
            nn.ReLU(),  # 활성화 함수 ReLU
            nn.Linear(4 * n_embd, n_embd), #multiplication performed in attention is all you need paper (다시 원래 차원(n_embd)으로 축소)
            # expands and contracts back down to projection
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

이 클래스는 트랜스포머 모델의 "피드포워드 신경망(Feed Forward Neural Network, FFN)"을 구현한 부분
- 입력 차원(n_embd) → 중간 차원(4 * n_embd) → 출력 차원(n_embd)

## 트랜스포머 블록

In [ ]:
class Block(nn.Module):

    def __init__(self, n_embd, n_head):
        super().__init__()
        head_size = n_embd // n_head
        # communication
        self.sa = MultiHeadAttention(n_head, head_size)
        # computation
        self.ffwd = FeedForward(n_embd)
        # layer norm
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

Multi-Head Self-Attention + Feed Forward Network + Layer Normalization 적용

## GPT 모델

In [ ]:
# create heart GPT class
class HeartGPTModel(nn.Module):

    def __init__(self):  
        super().__init__()
        # table needs to be vocab size by vocab size, to look up probability of next token given this token
        self.token_embedding_table = nn.Embedding(vocab_size,n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head = n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        #idx is batch, targets is time
        tok_emb = self.token_embedding_table(idx) #(B, T, vocab_size) which is batch, time, channel
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # T, C (integers from 0 to T-1)
        x = tok_emb + pos_emb # B, T, C
        x = self.blocks(x) # B, T, C
        x = self.ln_f(x) # B, T, C

        logits = self.lm_head(x)
        #channel is vocab size, so in this case 65

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx (context) to the last block_size tokens because positional embeddings only has up to block size
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

In [ ]:
model = HeartGPTModel()
m = model.to(device)
# random loss at this point would be -log(1/65)

#AdamW
optimizer  = torch.optim.AdamW(m.parameters(), lr=learning_rate)

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

# counter the number of model parameters to be trained
num_parameters = count_parameters(model)
print(f"The model has {num_parameters} trainable parameters.")

for iter in range(max_iters):
    
    if iter % eval_interval == 0:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    if iter % save_interval == 0:
        #model_path for checkpointing
        model_path = f"D:/ECGPT_pretrained_{n_embd}_{n_head}_{n_layer}_{block_size}_{max_iters}_{iter}.pth"
        torch.save(model.state_dict(), model_path)
    #get batch
    x_batch, y_batch = get_batch('train')

    # loss evaluation
    logits, loss = m(x_batch, y_batch)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()